In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery
from requests.adapters import HTTPAdapter, Retry

In [2]:
def get_links(page_num_nm, page_num_corona):
    # Extraction of all links till june 2020
    service = Service(ChromeDriverManager().install())


    #driver = webdriver.Chrome(service=service)
    browser = webdriver.Chrome(service=service)
    urls = []
    counter = 2
    articles_link = ["Links"]
    # since the tops category has 7 pages, where link to each follows a specific pattern identified above, we can create links to them as following:
    urls.append('https://newsmobile.in/articles/category/nm-fact-checker/')
    for i in range(2, page_num_nm):
        urls.append( 'https://newsmobile.in/articles/category/nm-fact-checker/page/' + str(counter) + '/')
        counter += 1

    counter = 2
    urls.append('https://newsmobile.in/articles/category/coronavirus-fact-check/')
    for i in range(2, page_num_corona):
        urls.append( 'https://newsmobile.in/articles/category/coronavirus-fact-check/page/' + str(counter) + '/')
        counter += 1

    for url in urls:
        # open the url
        browser.get(url)
        # purposeful wait time to allow the website to get fully loaded
        time.sleep(4)
        # get page content
        content = browser.page_source
        soup = BeautifulSoup(content, "lxml")
        art_links = []
        # extract all the anchor i.e., <a> elements with “thumb-link” class from the page
        data_links = soup.find_all("div", {"class": "td-module-thumb"})
        # from each <a> element, extract the URL
        for i in data_links:
            ia = i.find('a')
            art_links.append(ia['href'])

        articles_link.extend(art_links[5:-3])
        # purposeful wait time to avoid sending requests in quick succession
        time.sleep(10)
        
    browser.quit()
    
    np.savetxt("Newsmobile_english_links_june2020.csv", articles_link, delimiter =", ",fmt ='% s')
    
    return 

In [3]:
def get_content(dt):
    for url in dt["Links"]:
        #url = 'https://newsmobile.in/articles/2020/06/01/fake-alert-telangana-govt-has-not-released-any-guidelines-to-reopen-schools-in-the-state/'
        #response = requests.get(str(url))
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('property'):
                    meta_features[tag['property']] = tag.get('content')

            claim.append(meta_features['og:description'])
            # Published_date
            json_data = json.loads(soup.find(type="application/ld+json").string)
            Published_date.append(json_data['@graph'][0]['datePublished'])

            # title
            title.append(json_data['@graph'][0]['headline'])

            # top_image
            #top_image.append(json_data['@graph'][0]['image']['url'])

            # topic
            tags.append(json_data['@graph'][0]['articleSection'])

            # Content
            data = soup.find_all('p')
            text_content = []
            for paragraph in data:
                text_content.append(paragraph.get_text(strip=True))

            link = []
            for t in data:
                te = t.find('a')
                if te:
                    link.append(str(te['href']))

            links_in_text.append(link)

            # Join the extracted text content into a single string
            extracted_text = '\n'.join(text_content)

            Content.append(extracted_text)
            Linnnks.append(url)
            
    df = pd.DataFrame({'link':Linnnks, 'title': title, 'publish_date': Published_date, 'content': Content, 'claim': claim, 'Links_in_text':links_in_text,'tags':tags})
    df.to_csv('newsmobile_english_june2020.csv')
    return

In [5]:
if __name__ == "__main__":
    title = []
    Published_date = []
    Content = []
    video = []
    #top_image = []
    links_in_text = []
    tags = []
    claim = []
    Linnnks = []
    get_links(181, 23)
    dataframe = pd.read_csv("Newsmobile_english_links_june2020.csv")
    get_content(dataframe)